## The notebook combines the Surya bhawan csv files into one.

In [156]:
# Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from os import listdir
import math

### Clubbing all the csv files into One

In [63]:
# Directory location
file_dir = 'Solar_Data/Surya Bhawan/'

#Getting a list of all files
list_files = [f for f in listdir(file_dir) if 'csv' in f and 'lock' not in f]

# Now I have a list of all files

In [11]:
# Since all the csv's are in same format, therefore manually edited a test csv to extract the perimeters
test_csv = pd.read_excel(file_dir+'test.xlsx')
test_csv.head()

,TimeStamp,IntSolIrr,SMA-h-On,TmpMdul K,A.Ms.Amp,A.Ms.Vol,A.Ms.Watt,A1.Ms.Amp,B.Ms.Amp,B.Ms.Vol,...,GridMs.VA.phsC,GridMs.VAr.phsA,GridMs.VAr.phsB,GridMs.VAr.phsC,GridMs.W.phsA,GridMs.W.phsB,GridMs.W.phsC,Mode,Mt.TotOpTmh,Mt.TotTmh
0,hh:mm,W/m^2,h,K,A,V,W,A,A,V,...,VA,VAr,VAr,VAr,W,W,W,NaN,h,h
1,07:15,0,6066.37,282.56,0,595.51,0,0,0,594.86,...,0,0,0,0,0,0,0,7: Warten,3184.76,3474.33
2,07:30,5.29,6066.62,282.65,0.2,562.48,106.57,0.2,0.13,528.51,...,41.33,-14.53,-10.73,-12.07,40.33,39.4,39.73,1: Mpp,3184.94,3474.57
3,07:45,20.59,6066.87,283.46,0.64,590.18,379.03,0.64,0.41,589.37,...,188.45,-16.14,-12.69,-13.45,190.45,187.62,188.45,1: Mpp,3185.19,3474.82
4,08:00,45.47,6067.12,283.52,1.35,633.24,857.3,1.35,0.88,623.73,...,456.63,-8.47,-7.93,-8.93,455.63,454.63,456.43,1: Mpp,3185.44,3475.07


In [40]:
# Extracting the column names from this
Import_columns = test_csv.columns      # A list of all the columns in the test set

# Creating a dictionay of column names and their symbols using the row number 1
quan_symbol_dict = dict(zip(Import_columns, test_csv.loc[0].values))     # A dictionary consisting of the quantity and its particular unit
#quan_symbol_dict    # Unhash to see the dictionary

# Now removing the row containing the symbols
test_csv = test_csv[1:]

In [46]:
test_csv.head()

,TimeStamp,IntSolIrr,SMA-h-On,TmpMdul K,A.Ms.Amp,A.Ms.Vol,A.Ms.Watt,A1.Ms.Amp,B.Ms.Amp,B.Ms.Vol,...,GridMs.VA.phsC,GridMs.VAr.phsA,GridMs.VAr.phsB,GridMs.VAr.phsC,GridMs.W.phsA,GridMs.W.phsB,GridMs.W.phsC,Mode,Mt.TotOpTmh,Mt.TotTmh
1,07:15,0,6066.37,282.56,0,595.51,0,0,0,594.86,...,0,0,0,0,0,0,0,7: Warten,3184.76,3474.33
2,07:30,5.29,6066.62,282.65,0.2,562.48,106.57,0.2,0.13,528.51,...,41.33,-14.53,-10.73,-12.07,40.33,39.4,39.73,1: Mpp,3184.94,3474.57
3,07:45,20.59,6066.87,283.46,0.64,590.18,379.03,0.64,0.41,589.37,...,188.45,-16.14,-12.69,-13.45,190.45,187.62,188.45,1: Mpp,3185.19,3474.82
4,08:00,45.47,6067.12,283.52,1.35,633.24,857.3,1.35,0.88,623.73,...,456.63,-8.47,-7.93,-8.93,455.63,454.63,456.43,1: Mpp,3185.44,3475.07
5,08:15,75.97,6067.37,283.87,2.53,658.28,1667.24,2.53,1.65,648.01,...,895.03,-4,-3.66,-5.79,887.72,886.55,895.03,1: Mpp,3185.69,3475.32


In [263]:
def first_non_null(values_list):
    '''
    Function : Takes the column values as input and returns the first index with non null value
    
    '''
    for index, val in enumerate(values_list):
        if not math.isnan(val):
            return index
        
        
def last_non_null(values_list):
    '''
    Function : Takes the column values as input and returns the last index with non null value
    
    '''
    for index, val in enumerate(values_list):
        if math.isnan(val):
            return index


def csv_preprocess(file):
    '''
    Function : Applied data preprocessing on the given csv

    Input : Raw csv file path
    Output : Processed csv
    '''
    filepath = file_dir + file
    date = file[:-4]
    
    # Rows [1,2,3,5] are not useful so skipping them, they are also causing problems in importing the csvs.
    csvfile = pd.read_csv(filepath,encoding='cp1252', skiprows=[0,1,2,3,5])
    
    # Find the first column whose 'GridMs.Hz' is non-null
    non_null_row = first_non_null(csvfile['GridMs.Hz'].values)
    csvfile = csvfile[non_null_row:]
    #print(non_null_row)
    
    # Find the last column whose 'GridMs.Hz' is non-null
    non_null_row = last_non_null(csvfile['GridMs.Hz'].values)
    csvfile = csvfile[:non_null_row]
    #print(non_null_row)
    
    # Remove the unnecessary columns -> take only the columns selected in the test.csv
    csvfile = csvfile[Import_columns]
    
    csvfile.reset_index(drop=True, inplace=True)
    
    # Attach date which is the file name in this case, to the timestamp column
    csvfile['TimeStamp'] = csvfile.TimeStamp.apply(lambda x: date +' '+x)
    
    return csvfile

In [264]:
# Executing a loop which imports all the csv files and clubbs them all togther
data = pd.DataFrame(columns = test_csv.columns)
for file in list_files:
    data = data.append(csv_preprocess(file))

In [267]:
# data.to_csv('Compiled.csv', index=False)